In [62]:
#Andrea Burns
#Data Reader
import pandas as pd
import numpy as np
import itertools

In [63]:
#read each data frame in from my excel sheets
xls = pd.ExcelFile('DataDownload.xls')
df1 = pd.read_excel(xls, 'ACCESS')
df2 = pd.read_excel(xls, 'STORES')
df3 = pd.read_excel(xls, 'RESTAURANTS')
df4 = pd.read_excel(xls, 'ASSISTANCE')
df5 = pd.read_excel(xls, 'INSECURITY')
df6 = pd.read_excel(xls, 'PRICES_TAXES')
df7 = pd.read_excel(xls, 'LOCAL')
df8 = pd.read_excel(xls, 'HEALTH')
df9 = pd.read_excel(xls, 'SOCIOECONOMIC')

In [64]:
#these will be my output space; I'll try to predict these features
diabetes08 = df8['PCT_DIABETES_ADULTS08'].as_matrix()
diabetes13 = df8['PCT_DIABETES_ADULTS13'].as_matrix()
obesity08 = df8['PCT_OBESE_ADULTS08'].as_matrix()
obesity13 = df8['PCT_OBESE_ADULTS13'].as_matrix()

In [65]:
#remove my Y from the feature space to train on
df8 = df8.drop('PCT_DIABETES_ADULTS08', 1)
df8 = df8.drop('PCT_DIABETES_ADULTS13', 1)
df8 = df8.drop('PCT_OBESE_ADULTS08', 1)
df8 = df8.drop('PCT_OBESE_ADULTS13', 1)

In [66]:
alldata = [df1,df2,df3,df4,df5,df6,df7,df8,df9] 

In [67]:
#formatting all instances
X = []
i = 0
for i in range(0,3143):
    current_row = []
    for frame in alldata:
        mat = frame.as_matrix()
        current_row.append(mat[i][3:])
    X.append(list(itertools.chain(*current_row)))

In [68]:
#some features are missing data for specific instances, so I want to find them and deal with them
missing = np.argwhere(np.isnan(X))

In [69]:
feats = []
for i in range(0,len(missing)):
    feats.append(missing[i][1])
print(set(feats))

{1, 2, 4, 6, 7, 9, 11, 12, 14, 15, 16, 18, 19, 21, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 69, 70, 73, 76, 79, 82, 85, 88, 93, 94, 95, 99, 100, 101, 115, 116, 117, 118, 125, 126, 127, 128, 129, 130, 146, 147, 148, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 254, 257, 266, 267, 269, 272}


In [70]:
#these are two of the ~10 categorical features. Only these two have missing features
#since they are 0 or 1 binary classes, it doesn't make sense to replace NaN values with the mean of the column
#so instead I replace it will -1 as a "missing" class
for i in range(0,len(missing)):
    if missing[i][1] == 249:
        X[missing[i][0]][249] = -1
    elif missing[i][1] == 250:
        X[missing[i][0]][250] = -1

In [71]:
#all other features are continuous so I can impute NaN values with the mean of the feature!
from sklearn.preprocessing import Imputer
# missing_values is the value of your placeholder, strategy is if you'd like mean, median or mode, and axis=0 means it calculates the imputation based on the other feature values for that sample
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X)
inputspace = imp.transform(X)

In [89]:
#I didn't previously realize that the diabetes and obesity vectors had Nan 
#now fixing it similarly with the below cells, only a few were missing (3 or 4)
imp2 = Imputer(missing_values='NaN', strategy='mean', axis=0)
diabetes08=diabetes08.reshape(-1,1)
imp2.fit(diabetes08)
outputspace1 = imp2.transform(diabetes08)

In [90]:
imp3 = Imputer(missing_values='NaN', strategy='mean', axis=0)
diabetes13=diabetes13.reshape(-1,1)
imp3.fit(diabetes13)
outputspace2 = imp3.transform(diabetes13)

In [91]:
imp4 = Imputer(missing_values='NaN', strategy='mean', axis=0)
obesity08=obesity08.reshape(-1,1)
imp4.fit(obesity08)
outputspace3 = imp4.transform(obesity08)

In [92]:
imp5 = Imputer(missing_values='NaN', strategy='mean', axis=0)
obesity13=obesity13.reshape(-1,1)
imp5.fit(obesity13)
outputspace4 = imp5.transform(obesity13)

In [118]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as prep
inputspace = prep.scale(inputspace)

#making different classifiers to try to predict the four rates 
X_train, X_test, y_train, y_test = train_test_split(inputspace,outputspace1.ravel(), test_size=0.1, random_state=42)
diab8clf = SVR(C=1.0, epsilon=0.2)

X_train2, X_test2, y_train2, y_test2 = train_test_split(inputspace,outputspace2.ravel(), test_size=0.1, random_state=42)
diab13clf = SVR(C=1.0, epsilon=0.2)

X_train3, X_test3, y_train3, y_test3 = train_test_split(inputspace,outputspace3.ravel(), test_size=0.1, random_state=42)
obes8clf = SVR(C=1.0, epsilon=0.2)

X_train4, X_test4, y_train4, y_test4 = train_test_split(inputspace,outputspace4.ravel(), test_size=0.1, random_state=42)
obes13clf = SVR(C=1.0, epsilon=0.2)

In [119]:
#fitting train data with SVM model for regression, as I am predicting the diabetes or obesity rates
#which are continuous values
diab8clf.fit(X_train,y_train)
diab13clf.fit(X_train2,y_train2)
obes8clf.fit(X_train3,y_train3)
obes13clf.fit(X_train4,y_train4)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [120]:
#test portion 
diabetes8pred = diab8clf.predict(X_test)
diabetes13pred = diab13clf.predict(X_test2)
obesity8pred = obes8clf.predict(X_test3)
obesity13pred = obes13clf.predict(X_test4)

In [122]:
#these are two standard performance metrics for regression, printing them for each
#mean squared error has been shown in class, kind of like avg. distance btw output and target values
#pearson coefficient is a statistic used to show the strength of a linear relationship
#best pearson value possible is -1 or 1 (perfectly negative or positive relationship)

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
print('Pearson Coefficient and MSE for Diabetes Prediction in 2008')
print(pearsonr(np.asarray(y_test).ravel(),diabetes8pred.ravel()))
mean_sqr1 = mean_squared_error(diabetes8pred, y_test)
print(mean_sqr1)
print()

print('Pearson Coefficient and MSE for Diabetes Prediction in 2013')
print(pearsonr(np.asarray(y_test2).ravel(),diabetes13pred.ravel()))
mean_sqr2 = mean_squared_error(diabetes13pred, y_test2)
print(mean_sqr2)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2008')
print(pearsonr(np.asarray(y_test3).ravel(),obesity8pred.ravel()))
mean_sqr3 = mean_squared_error(obesity8pred, y_test3)
print(mean_sqr3)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2013')
print(pearsonr(np.asarray(y_test4).ravel(),obesity13pred.ravel()))
mean_sqr4 = mean_squared_error(obesity13pred, y_test4)
print(mean_sqr4)

Pearson Coefficient and MSE for Diabetes Prediction in 2008
(0.9119756128617906, 3.7663255632562475e-123)
0.82236275945728

Pearson Coefficient and MSE for Diabetes Prediction in 2013
(0.8740210255469344, 3.960268806775074e-100)
1.5043250592260557

Pearson Coefficient and MSE for Obesity Prediction in 2008
(0.8363386149441886, 1.052059362694424e-83)
4.542912274967804

Pearson Coefficient and MSE for Obesity Prediction in 2013
(0.8165881401494428, 1.0994118503700413e-76)
7.158954486310911
